In [3]:
import numpy as np
import geopy
import pandas as pd
import math
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
search = SearchEngine()
import glob
import googlemaps

In [2]:
PATH = '/Users/richardryu/Desktop/Cantwo/w210/data/*.csv'

In [20]:
gmaps = googlemaps.Client(key = 'your API KEY')

In [5]:
pm = pd.read_csv('parking_meters.csv')
# class 0- lamp, class 1- sign-h, class 2- fh, class 3-nopark, class 4-stop, class 5- pm
pm_ff3 = pd.DataFrame(columns=['lat', 'long', 'class'])
for i in range(len(pm)):
    result = search.by_coordinates(pm.loc[i, 'LATITUDE'], pm.loc[i, 'LONGITUDE'], radius=2)
    pm_ff3.loc[i, 'lat'] = pm.loc[i, 'LATITUDE']
    pm_ff3.loc[i, 'long'] = pm.loc[i, 'LONGITUDE']
    pm_ff3.loc[i, 'class'] = 5

In [149]:
# 7/1 save df by zipcode in csv
def zip_df(df):
    for i in df["zipcode"].unique():
        temp_df = df[df["zipcode"] == i]
        temp_df.to_csv(i + '.csv', encoding = 'utf-8', index = False)

In [6]:
# lat long transformer
def csv_par11(df, delta):
    for i in range(len(df)):
        df.loc[i, 'lat'] = float(df.loc[i, 'imgname'].split('-')[0])
        df.loc[i, 'long'] = float(df.loc[i, 'imgname'].split('-')[2]) * -1.0
        df.loc[i, 'angle'] = df.loc[i, 'imgname'].split('-')[3][:-4]


        if df.loc[i, 'xcenter'] <= 0.333333:
            df.loc[i, 'detect_loc'] = 'L'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat'] + (delta * math.cos((math.pi / 180)*(float(df.loc[i, 'angle'])-90)))
            df.loc[i, 'new_long'] = df.loc[i, 'long'] + (delta * math.sin((math.pi / 180)*(float(df.loc[i, 'angle'])-90)))
        elif df.loc[i, 'xcenter'] > 0.666666:
            df.loc[i, 'detect_loc'] = 'R'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat'] - (delta * math.cos((math.pi / 180)*(float(df.loc[i, 'angle']) -90)))
            df.loc[i, 'new_long'] = df.loc[i, 'long'] - (delta * math.sin((math.pi / 180)*(float(df.loc[i, 'angle']) -90)))
        else:
            df.loc[i, 'detect_loc'] = 'C'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat']
            df.loc[i, 'new_long'] = df.loc[i, 'long']
    return df

In [8]:
# grabbing all csv files and reading them as separate pd dataframes
temp = []
for fname in glob.glob(PATH):
    temp.append(pd.read_csv(fname))

In [9]:
# transforming lat/long on each csv file and adding new lat/long with 2 zipcodes
conv_temp = []
for i in temp:
    conv_temp.append(csv_par11(i, 0.0001))

In [12]:
final_df13 = pd.concat([conv_temp[0], conv_temp[1], conv_temp[2], conv_temp[3], conv_temp[4], conv_temp[5], conv_temp[6], conv_temp[7], conv_temp[8], conv_temp[9], conv_temp[10], conv_temp[11], conv_temp[12], conv_temp[13], conv_temp[14]] , ignore_index=True)

In [14]:
# dropping "className" which is filled with NaN and is a duplicate of 'class'
new_final_df13 = final_df13.drop(columns = ["className"])
# filt_fin_df13 = new_final_df13[new_final_df13["confidence"] > 0.5]
mosaic9 = pd.concat([new_final_df13, pm_ff3], ignore_index=True)

In [13]:
final_df13

,imgname,lat,long,angle,class,className,confidence,xcenter,ycenter,width,hight,detect_loc,new_lat,new_long
0,39.67933178--105.0459518-270.jpg,39.679332,-105.045952,270,2,fh,0.669755,0.701563,0.637500,0.021875,0.037500,R,39.679432,-105.045952
1,39.66952458--104.91756090000001-270.jpg,39.669525,-104.917561,270,2,fh,0.736712,0.731250,0.685156,0.028125,0.042188,R,39.669625,-104.917561
2,39.67047403--105.0358949-270.jpg,39.670474,-105.035895,270,0,lamp,0.854703,0.298438,0.503906,0.106250,0.392188,L,39.670374,-105.035895
3,39.68569746--104.9919376-270.jpg,39.685697,-104.991938,270,2,fh,0.465201,0.350781,0.680469,0.026562,0.060937,C,39.685697,-104.991938
4,39.71416001--105.0457517-270.jpg,39.714160,-105.045752,270,0,lamp,0.814633,0.407031,0.521875,0.073438,0.265625,C,39.714160,-105.045752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39972,39.72610402--105.0324926-90.jpg,39.726104,-105.032493,90,0,NaN,0.665202,0.963281,0.485156,0.070312,0.410937,R,39.726004,-105.032493
39973,39.72610402--105.0324926-90.jpg,39.726104,-105.032493,90,0,NaN,0.484138,0.475000,0.535937,0.025000,0.153125,C,39.726104,-105.032493
39974,39.73053788--105.033289-90.jpg,39.730538,-105.033289,90,0,NaN,0.828780,0.697656,0.372656,0.089063,0.689062,R,39.730438,-105.033289
39975,39.74034077--105.0364632-90.jpg,39.740341,-105.036463,90,0,NaN,0.716447,0.746094,0.532031,0.045312,0.176563,R,39.740241,-105.036463


In [15]:
mosaic9

,imgname,lat,long,angle,class,confidence,xcenter,ycenter,width,hight,detect_loc,new_lat,new_long
0,39.67933178--105.0459518-270.jpg,39.6793,-105.046,270,2,0.669755,0.701563,0.637500,0.021875,0.037500,R,39.679432,-105.045952
1,39.66952458--104.91756090000001-270.jpg,39.6695,-104.918,270,2,0.736712,0.731250,0.685156,0.028125,0.042188,R,39.669625,-104.917561
2,39.67047403--105.0358949-270.jpg,39.6705,-105.036,270,0,0.854703,0.298438,0.503906,0.106250,0.392188,L,39.670374,-105.035895
3,39.68569746--104.9919376-270.jpg,39.6857,-104.992,270,2,0.465201,0.350781,0.680469,0.026562,0.060937,C,39.685697,-104.991938
4,39.71416001--105.0457517-270.jpg,39.7142,-105.046,270,0,0.814633,0.407031,0.521875,0.073438,0.265625,C,39.714160,-105.045752
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46116,NaN,39.7431,-104.994,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46117,NaN,39.7515,-105,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46118,NaN,39.7393,-105.007,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46119,NaN,39.7317,-104.935,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
mosaic9['lat'][0]

39.67933178

In [19]:
lat1 = mosaic9['lat'][0]
long1 = mosaic9['long'][0]

In [21]:
reverse_geocode_result = gmaps.reverse_geocode((lat1, long1), result_type = "postal_code")

In [60]:
reverse_geocode_result[0].items()

dict_items([('address_components', [{'long_name': '80219', 'short_name': '80219', 'types': ['postal_code']}, {'long_name': 'Denver', 'short_name': 'Denver', 'types': ['locality', 'political']}, {'long_name': 'Denver County', 'short_name': 'Denver County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Colorado', 'short_name': 'CO', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}]), ('formatted_address', 'Denver, CO 80219, USA'), ('geometry', {'bounds': {'northeast': {'lat': 39.72388309999999, 'lng': -105.014073}, 'southwest': {'lat': 39.6662149, 'lng': -105.0532781}}, 'location': {'lat': 39.6961606, 'lng': -105.0382076}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 39.72388309999999, 'lng': -105.014073}, 'southwest': {'lat': 39.6662149, 'lng': -105.0532781}}}), ('place_id', 'ChIJnXP59oCAa4cRIhZCLCt0jxU'), ('types', ['postal_code'])])

In [63]:
reverse_geocode_result[0].get('address_components')[0].get('long_name')

'80219'

In [36]:
for i in list(range(len(mosaic9))):
    mosaic9.loc[i, 'coord'] = str(mosaic9.loc[i, 'lat']) + str(mosaic9.loc[i, 'long'])

In [38]:
western = mosaic9['coord']

In [42]:
eastern = western.drop_duplicates()

In [43]:
eastern

0                      39.67933178-105.0459518
1               39.66952458-104.91756090000001
2                      39.67047403-105.0358949
3                      39.68569746-104.9919376
4                      39.71416001-105.0457517
                         ...                  
46116              39.74309-104.99418200000001
46117    39.751529999999995-104.99973999999999
46118                     39.739257-105.007103
46119                     39.731651-104.935259
46120                      39.739899-104.96394
Name: coord, Length: 33636, dtype: object

In [44]:
eastern = mosaic9.drop_duplicates(subset='coord', keep="first")

In [45]:
eastern

,imgname,lat,long,angle,class,confidence,xcenter,ycenter,width,hight,detect_loc,new_lat,new_long,coord
0,39.67933178--105.0459518-270.jpg,39.6793,-105.046,270,2,0.669755,0.701563,0.637500,0.021875,0.037500,R,39.679432,-105.045952,39.67933178-105.0459518
1,39.66952458--104.91756090000001-270.jpg,39.6695,-104.918,270,2,0.736712,0.731250,0.685156,0.028125,0.042188,R,39.669625,-104.917561,39.66952458-104.91756090000001
2,39.67047403--105.0358949-270.jpg,39.6705,-105.036,270,0,0.854703,0.298438,0.503906,0.106250,0.392188,L,39.670374,-105.035895,39.67047403-105.0358949
3,39.68569746--104.9919376-270.jpg,39.6857,-104.992,270,2,0.465201,0.350781,0.680469,0.026562,0.060937,C,39.685697,-104.991938,39.68569746-104.9919376
4,39.71416001--105.0457517-270.jpg,39.7142,-105.046,270,0,0.814633,0.407031,0.521875,0.073438,0.265625,C,39.714160,-105.045752,39.71416001-105.0457517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46116,NaN,39.7431,-104.994,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.74309-104.99418200000001
46117,NaN,39.7515,-105,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.751529999999995-104.99973999999999
46118,NaN,39.7393,-105.007,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.739257-105.007103
46119,NaN,39.7317,-104.935,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.731651-104.935259


In [48]:
pho_tasty = pd.DataFrame([eastern['coord'], eastern['new_lat'], eastern['new_long']])

In [50]:
pho_tasty.T

,coord,new_lat,new_long
0,39.67933178-105.0459518,39.6794,-105.046
1,39.66952458-104.91756090000001,39.6696,-104.918
2,39.67047403-105.0358949,39.6704,-105.036
3,39.68569746-104.9919376,39.6857,-104.992
4,39.71416001-105.0457517,39.7142,-105.046
...,...,...,...
46116,39.74309-104.99418200000001,NaN,NaN
46117,39.751529999999995-104.99973999999999,NaN,NaN
46118,39.739257-105.007103,NaN,NaN
46119,39.731651-104.935259,NaN,NaN


In [51]:
kentucky = pho_tasty.T.reset_index(drop=True)

In [55]:
kentucky

,coord,new_lat,new_long
0,39.67933178-105.0459518,39.6794,-105.046
1,39.66952458-104.91756090000001,39.6696,-104.918
2,39.67047403-105.0358949,39.6704,-105.036
3,39.68569746-104.9919376,39.6857,-104.992
4,39.71416001-105.0457517,39.7142,-105.046
...,...,...,...
33631,39.74309-104.99418200000001,NaN,NaN
33632,39.751529999999995-104.99973999999999,NaN,NaN
33633,39.739257-105.007103,NaN,NaN
33634,39.731651-104.935259,NaN,NaN


## POC: Google Maps Reverse Geocoding testing on 10 rows of DF

In [75]:
david = kentucky[:10]

In [76]:
david

,coord,new_lat,new_long
0,39.67933178-105.0459518,39.6794,-105.046
1,39.66952458-104.91756090000001,39.6696,-104.918
2,39.67047403-105.0358949,39.6704,-105.036
3,39.68569746-104.9919376,39.6857,-104.992
4,39.71416001-105.0457517,39.7142,-105.046
5,39.71572973-105.02016189999999,39.7158,-105.02
6,39.68182538-104.9409663,39.6817,-104.941
7,39.67853634-105.0203696,39.6786,-105.02
8,39.71775385-104.973395,39.7177,-104.973
9,39.69050769-104.912888,39.6906,-104.913


In [78]:
for i in list(range(len(david))):
    temp_result = gmaps.reverse_geocode((david.loc[i,'new_lat'], david.loc[i, 'new_long']), result_type = "postal_code")
    david.loc[i, 'zipcode'] = temp_result[0].get('address_components')[0].get('long_name')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is try

In [79]:
david

,coord,new_lat,new_long,zipcode
0,39.67933178-105.0459518,39.6794,-105.046,80219
1,39.66952458-104.91756090000001,39.6696,-104.918,80222
2,39.67047403-105.0358949,39.6704,-105.036,80219
3,39.68569746-104.9919376,39.6857,-104.992,80223
4,39.71416001-105.0457517,39.7142,-105.046,80219
5,39.71572973-105.02016189999999,39.7158,-105.02,80219
6,39.68182538-104.9409663,39.6817,-104.941,80210
7,39.67853634-105.0203696,39.6786,-105.02,80219
8,39.71775385-104.973395,39.7177,-104.973,80218
9,39.69050769-104.912888,39.6906,-104.913,80224


In [81]:
for i in list(range(len(kentucky))):
    temp_result = gmaps.reverse_geocode((kentucky.loc[i,'new_lat'], kentucky.loc[i, 'new_long']), result_type = "postal_code")
    kentucky.loc[i, 'zipcode'] = temp_result[0].get('address_components')[0].get('long_name')

HTTPError: HTTP Error: 400

### API timed out above, so manually finding where the job got cutoff in the ordered pd DF...

In [82]:
kentucky

,coord,new_lat,new_long,zipcode
0,39.67933178-105.0459518,39.6794,-105.046,80219
1,39.66952458-104.91756090000001,39.6696,-104.918,80222
2,39.67047403-105.0358949,39.6704,-105.036,80219
3,39.68569746-104.9919376,39.6857,-104.992,80223
4,39.71416001-105.0457517,39.7142,-105.046,80219
...,...,...,...,...
33631,39.74309-104.99418200000001,NaN,NaN,NaN
33632,39.751529999999995-104.99973999999999,NaN,NaN,NaN
33633,39.739257-105.007103,NaN,NaN,NaN
33634,39.731651-104.935259,NaN,NaN,NaN


In [107]:
kentucky.loc[27501]

coord       39.74265866-105.04980619999999
new_lat                            39.7427
new_long                           -105.05
zipcode                              80204
Name: 27501, dtype: object

In [ ]:
kentucky["new_lat"] == np.isnan

In [108]:
33636-27502

6134

In [109]:
len(pm_ff3)

6144

In [110]:
kentucky.loc[27502]

coord       39.739492999999996-104.982258
new_lat                               NaN
new_long                              NaN
zipcode                               NaN
Name: 27502, dtype: object

In [121]:
chicken1 = kentucky[:27502]

In [122]:
chicken1

,coord,new_lat,new_long,zipcode
0,39.67933178-105.0459518,39.6794,-105.046,80219
1,39.66952458-104.91756090000001,39.6696,-104.918,80222
2,39.67047403-105.0358949,39.6704,-105.036,80219
3,39.68569746-104.9919376,39.6857,-104.992,80223
4,39.71416001-105.0457517,39.7142,-105.046,80219
...,...,...,...,...
27497,39.74001395-104.99248349999999,39.74,-104.992,80204
27498,39.74025271-105.01057720000001,39.7403,-105.011,80204
27499,39.73053788-105.033289,39.7304,-105.033,80204
27500,39.74034077-105.0364632,39.7402,-105.036,80204


In [111]:
chicken2 = kentucky[27502:]

In [112]:
chicken2

,coord,new_lat,new_long,zipcode
27502,39.739492999999996-104.982258,NaN,NaN,NaN
27503,39.740076-104.957586,NaN,NaN,NaN
27504,39.730996999999995-104.98728200000001,NaN,NaN,NaN
27505,39.743344-104.98286499999999,NaN,NaN,NaN
27506,39.735202-104.989418,NaN,NaN,NaN
...,...,...,...,...
33631,39.74309-104.99418200000001,NaN,NaN,NaN
33632,39.751529999999995-104.99973999999999,NaN,NaN,NaN
33633,39.739257-105.007103,NaN,NaN,NaN
33634,39.731651-104.935259,NaN,NaN,NaN


In [115]:
pho_tasty = pd.DataFrame([eastern['coord'], eastern['lat'], eastern['long']])

In [116]:
fried_c = pho_tasty.T[27502:].reset_index(drop=True)

In [117]:
fried_c

,coord,lat,long
0,39.739492999999996-104.982258,39.7395,-104.982
1,39.740076-104.957586,39.7401,-104.958
2,39.730996999999995-104.98728200000001,39.731,-104.987
3,39.743344-104.98286499999999,39.7433,-104.983
4,39.735202-104.989418,39.7352,-104.989
...,...,...,...
6129,39.74309-104.99418200000001,39.7431,-104.994
6130,39.751529999999995-104.99973999999999,39.7515,-105
6131,39.739257-105.007103,39.7393,-105.007
6132,39.731651-104.935259,39.7317,-104.935


In [118]:
for i in list(range(len(fried_c))):
    temp_result = gmaps.reverse_geocode((fried_c.loc[i,'lat'], fried_c.loc[i, 'long']), result_type = "postal_code")
    fried_c.loc[i, 'zipcode'] = temp_result[0].get('address_components')[0].get('long_name')

In [119]:
fried_c

,coord,lat,long,zipcode
0,39.739492999999996-104.982258,39.7395,-104.982,80203
1,39.740076-104.957586,39.7401,-104.958,80206
2,39.730996999999995-104.98728200000001,39.731,-104.987,80203
3,39.743344-104.98286499999999,39.7433,-104.983,80203
4,39.735202-104.989418,39.7352,-104.989,80204
...,...,...,...,...
6129,39.74309-104.99418200000001,39.7431,-104.994,80202
6130,39.751529999999995-104.99973999999999,39.7515,-105,80202
6131,39.739257-105.007103,39.7393,-105.007,80204
6132,39.731651-104.935259,39.7317,-104.935,80220


In [120]:
fried_c.to_csv('friedc.csv', encoding = 'utf-8', index = False)

In [123]:
chicken1.to_csv('chicken1.csv', encoding = 'utf-8', index = False)

In [125]:
chicken1

,coord,new_lat,new_long,zipcode
0,39.67933178-105.0459518,39.6794,-105.046,80219
1,39.66952458-104.91756090000001,39.6696,-104.918,80222
2,39.67047403-105.0358949,39.6704,-105.036,80219
3,39.68569746-104.9919376,39.6857,-104.992,80223
4,39.71416001-105.0457517,39.7142,-105.046,80219
...,...,...,...,...
27497,39.74001395-104.99248349999999,39.74,-104.992,80204
27498,39.74025271-105.01057720000001,39.7403,-105.011,80204
27499,39.73053788-105.033289,39.7304,-105.033,80204
27500,39.74034077-105.0364632,39.7402,-105.036,80204


In [127]:
fried_c

,coord,lat,long,zipcode
0,39.739492999999996-104.982258,39.7395,-104.982,80203
1,39.740076-104.957586,39.7401,-104.958,80206
2,39.730996999999995-104.98728200000001,39.731,-104.987,80203
3,39.743344-104.98286499999999,39.7433,-104.983,80203
4,39.735202-104.989418,39.7352,-104.989,80204
...,...,...,...,...
6129,39.74309-104.99418200000001,39.7431,-104.994,80202
6130,39.751529999999995-104.99973999999999,39.7515,-105,80202
6131,39.739257-105.007103,39.7393,-105.007,80204
6132,39.731651-104.935259,39.7317,-104.935,80220


In [135]:
g_zip = pd.concat([fried_c, chicken1], ignore_index=True)

In [136]:
g_zip

,coord,lat,long,zipcode,new_lat,new_long
0,39.739492999999996-104.982258,39.7395,-104.982,80203,NaN,NaN
1,39.740076-104.957586,39.7401,-104.958,80206,NaN,NaN
2,39.730996999999995-104.98728200000001,39.731,-104.987,80203,NaN,NaN
3,39.743344-104.98286499999999,39.7433,-104.983,80203,NaN,NaN
4,39.735202-104.989418,39.7352,-104.989,80204,NaN,NaN
...,...,...,...,...,...,...
33631,39.74001395-104.99248349999999,NaN,NaN,80204,39.74,-104.992
33632,39.74025271-105.01057720000001,NaN,NaN,80204,39.7403,-105.011
33633,39.73053788-105.033289,NaN,NaN,80204,39.7304,-105.033
33634,39.74034077-105.0364632,NaN,NaN,80204,39.7402,-105.036


In [137]:
new_zip = pd.DataFrame([g_zip['coord'], g_zip['zipcode']])

In [140]:
coord_tab = new_zip.T

In [129]:
mosaic9

,imgname,lat,long,angle,class,confidence,xcenter,ycenter,width,hight,detect_loc,new_lat,new_long,coord
0,39.67933178--105.0459518-270.jpg,39.6793,-105.046,270,2,0.669755,0.701563,0.637500,0.021875,0.037500,R,39.679432,-105.045952,39.67933178-105.0459518
1,39.66952458--104.91756090000001-270.jpg,39.6695,-104.918,270,2,0.736712,0.731250,0.685156,0.028125,0.042188,R,39.669625,-104.917561,39.66952458-104.91756090000001
2,39.67047403--105.0358949-270.jpg,39.6705,-105.036,270,0,0.854703,0.298438,0.503906,0.106250,0.392188,L,39.670374,-105.035895,39.67047403-105.0358949
3,39.68569746--104.9919376-270.jpg,39.6857,-104.992,270,2,0.465201,0.350781,0.680469,0.026562,0.060937,C,39.685697,-104.991938,39.68569746-104.9919376
4,39.71416001--105.0457517-270.jpg,39.7142,-105.046,270,0,0.814633,0.407031,0.521875,0.073438,0.265625,C,39.714160,-105.045752,39.71416001-105.0457517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46116,NaN,39.7431,-104.994,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.74309-104.99418200000001
46117,NaN,39.7515,-105,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.751529999999995-104.99973999999999
46118,NaN,39.7393,-105.007,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.739257-105.007103
46119,NaN,39.7317,-104.935,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.731651-104.935259


In [141]:
mosaic_copy = mosaic9.copy()

In [144]:
join_mosaic = mosaic_copy.join(coord_tab.set_index('coord'), on='coord')

In [145]:
join_mosaic

,imgname,lat,long,angle,class,confidence,xcenter,ycenter,width,hight,detect_loc,new_lat,new_long,coord,zipcode
0,39.67933178--105.0459518-270.jpg,39.6793,-105.046,270,2,0.669755,0.701563,0.637500,0.021875,0.037500,R,39.679432,-105.045952,39.67933178-105.0459518,80219
1,39.66952458--104.91756090000001-270.jpg,39.6695,-104.918,270,2,0.736712,0.731250,0.685156,0.028125,0.042188,R,39.669625,-104.917561,39.66952458-104.91756090000001,80222
2,39.67047403--105.0358949-270.jpg,39.6705,-105.036,270,0,0.854703,0.298438,0.503906,0.106250,0.392188,L,39.670374,-105.035895,39.67047403-105.0358949,80219
3,39.68569746--104.9919376-270.jpg,39.6857,-104.992,270,2,0.465201,0.350781,0.680469,0.026562,0.060937,C,39.685697,-104.991938,39.68569746-104.9919376,80223
4,39.71416001--105.0457517-270.jpg,39.7142,-105.046,270,0,0.814633,0.407031,0.521875,0.073438,0.265625,C,39.714160,-105.045752,39.71416001-105.0457517,80219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46116,NaN,39.7431,-104.994,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.74309-104.99418200000001,80202
46117,NaN,39.7515,-105,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.751529999999995-104.99973999999999,80202
46118,NaN,39.7393,-105.007,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.739257-105.007103,80204
46119,NaN,39.7317,-104.935,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.731651-104.935259,80220


In [150]:
zip_df(join_mosaic)

### Cutting on 0.5 conf Threshold

In [147]:
join_mosaic_05 = join_mosaic[join_mosaic['confidence'] > 0.5]

In [148]:
join_mosaic_05

,imgname,lat,long,angle,class,confidence,xcenter,ycenter,width,hight,detect_loc,new_lat,new_long,coord,zipcode
0,39.67933178--105.0459518-270.jpg,39.6793,-105.046,270,2,0.669755,0.701563,0.637500,0.021875,0.037500,R,39.679432,-105.045952,39.67933178-105.0459518,80219
1,39.66952458--104.91756090000001-270.jpg,39.6695,-104.918,270,2,0.736712,0.731250,0.685156,0.028125,0.042188,R,39.669625,-104.917561,39.66952458-104.91756090000001,80222
2,39.67047403--105.0358949-270.jpg,39.6705,-105.036,270,0,0.854703,0.298438,0.503906,0.106250,0.392188,L,39.670374,-105.035895,39.67047403-105.0358949,80219
4,39.71416001--105.0457517-270.jpg,39.7142,-105.046,270,0,0.814633,0.407031,0.521875,0.073438,0.265625,C,39.714160,-105.045752,39.71416001-105.0457517,80219
5,39.71572973--105.02016189999999-270.jpg,39.7157,-105.02,270,4,0.743508,0.813281,0.583594,0.039062,0.051562,R,39.715830,-105.020162,39.71572973-105.02016189999999,80219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39971,39.72610402--105.0324926-90.jpg,39.7261,-105.032,90,0,0.918574,0.084375,0.357812,0.165625,0.712500,L,39.726204,-105.032493,39.72610402-105.0324926,80204
39972,39.72610402--105.0324926-90.jpg,39.7261,-105.032,90,0,0.665202,0.963281,0.485156,0.070312,0.410937,R,39.726004,-105.032493,39.72610402-105.0324926,80204
39974,39.73053788--105.033289-90.jpg,39.7305,-105.033,90,0,0.828780,0.697656,0.372656,0.089063,0.689062,R,39.730438,-105.033289,39.73053788-105.033289,80204
39975,39.74034077--105.0364632-90.jpg,39.7403,-105.036,90,0,0.716447,0.746094,0.532031,0.045312,0.176563,R,39.740241,-105.036463,39.74034077-105.0364632,80204


In [151]:
zip_df(join_mosaic_05)